# F2PY basics
---
- Author: Diego Inácio
- GitHub: [github.com/diegoinacio](https://github.com/diegoinacio)
- Notebook: [_basics_F2PY.ipynb](https://github.com/diegoinacio/computer-vision-notebooks/blob/master/High-Performance-Computing/_basics_F2PY.ipynb)
- Spec: **CPU** i7-9750H 2.6 GHz **GPU** GeForce RTX 2060
---
High performance on *Python* and *NumPy* using [F2PY](https://numpy.org/devdocs/f2py/index.html).

In [1]:
import numpy as np

# 0. Installation
---

F2PY is a native *Fortran* to *Python* interface generator from [NumPy](https://numpy.org/). Thus, the [Installation](https://numpy.org/devdocs/user/install.html) command for *anaconda* and *pip*:

```
$ conda install numpy
```

or

```
$ pip install numpy
```

# 1. Compilation
---
A *Fortran* source file can have the name of the module. For example, given the source file `examples_f2py.f90` with a simple function which returns a string in bytes.

```fortran
function hello_f2py() result(output)
    character(len=12) :: output
    output = "Hello, F2PY!"
end function hello_f2py
```

The following step is the compilation, which is done by running the command:

```
$ f2py -m examples_f2py -c examples_f2py.f90
```

In [2]:
from examples_f2py import hello_f2py

In [3]:
print(hello_f2py())

b'Hello, F2PY!'


# 2. Performance
---
The following example, we will try to approximate the value $\large\pi$ with the idea of $\\tan^{-1}1=\frac{\pi}{4}$ using the power series of *arctan*, defined by:

$$\large
4 \sum_{n=0}^{N}\frac{(-1)^n}{2n+1} \approx \pi
$$

where $N$ tends to the infinity.

In [4]:
def pi_py(N):
    pi = 0
    for n in range(N):
        pi += (-1)**n/(2*n + 1)
    return 4*pi

print(pi_py(1000000))

3.1415916535897743


In the same *Fortran* source file `examples_f2py.f90`, lets include the function and adapt it to be compiled.

```fortran
function pi_f2py(N_) result(pi)
    integer :: N_, n
    real    :: pi
    pi = 0
    do n=0,N_
        pi = pi + (-1)**n/real(2*n + 1)
    end do
    pi = 4*pi
end function pi_f2py
```

*p.s.: compile it again by running the command:*

```
$ f2py -m examples_f2py -c examples_f2py.f90
```

In [5]:
from examples_f2py import pi_f2py

In [6]:
# Time measurement over the situations
print('[Python] pi_py |', end=' ')
%timeit -n 5 -r 5 pi_py(1000000)
print('[F2PY] pi_f2py |', end=' ')
%timeit -n 5 -r 5 pi_f2py(1000000)

[Python] pi_py | 576 ms ± 3.33 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
[F2PY] pi_f2py | 1.02 ms ± 11.1 µs per loop (mean ± std. dev. of 5 runs, 5 loops each)


# 3. Inline fortran source
---
Build inline fortran commands and compile with `numpy.f2py`.

In [7]:
fsource = '''
recursive function factorial(x) result(output)
    integer :: x, output
    if (x <= 1) then
        output = 1
    else
        output = x*factorial(x - 1)
    end if
end function factorial
'''

import numpy.f2py
numpy.f2py.compile(
    fsource, modulename='example_f2py', extension='.f90', verbose=0
)

0

In [8]:
from example_f2py import factorial
print(factorial(10))

3628800
